In [ ]:
import psycopg2
import urllib
from pathlib import Path
import re
import pandas as pd
from sqlalchemy import create_engine
import json

In [ ]:
# change connection 
conn = psycopg2.connect(
    host="localhost",
    database="payne_logs",
    user="zack")
cur = conn.cursor()

In [ ]:
def download(url_str: str, file_name: str, overwrite: bool=False):
    '''Downloads file'''
    file = Path(file_name)

    if not file.is_file() or overwrite:
        new_file = urllib.request.urlretrieve(url_str, file_name)

In [ ]:
log_files = [
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20200705",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20200712",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20200719",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20200726",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20200809",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20200816",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20200823",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20200830",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20200906",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20200913",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20200920",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20200927",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20201004",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20201011",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20201018",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20201025",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20201101",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20201108",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20201115",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20201122",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20201129",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20201206",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20201213",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20201220",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20201227",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210103",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210110",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210117",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210124",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210131",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210207",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210214",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210221",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210228",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210307",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210314",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210321",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210328",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210404",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210411",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210418",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210425",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210502",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210509",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210516",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210523",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210530",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210606",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210613",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210620",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210627",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210704",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210711",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210718",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210725",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210801",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210808",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210815",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210822",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210829",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210905",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210912",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210919",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20210926",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20211003",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20211010",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20211017",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20211024",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20211031",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20211107",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20211114",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20211121",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20211128",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20211205",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20211212",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20211219",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20211226",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20220102",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20220109",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20220116",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20220123",
"https://eogdata.mines.edu/wwwdata/hidden/eog_access_analysis/ssl_access_log-20220130",
]

In [ ]:
for i, url in enumerate(log_files):
    print(f'Downloading log: {i}')
    log_name = f'log{i}'
    download(url, log_name)
    # only keep certain filetypes with http success (200)
    !grep "\.zip\|\.gz\|\.tgz\|\.tif\|\.csv\|\.kmz" {log_name} | grep " 200 \([0-9]\+\|-\)\$" | sponge {log_name}

In [ ]:
def apache_log_to_sql(file_name: str, database_conn, overwrite: bool = False):
    '''parses accesses from log file and creates sqlite3 database file'''
    
    database_cur = database_conn.cursor()

    if overwrite:
        database_cur.execute("DROP TABLE IF EXISTS logs")

    database_cur.execute('''CREATE TABLE IF NOT EXISTS logs
               (time TIMESTAMPTZ, ip inet, userid text, file text, size bigint)''')

    with open(file_name) as log_file:
        for line in log_file:
            # ip, id, userid, timestamp, url, response, size
            tup = re.match(r'([(\d\.)]+) ([^\s]+) ([^\s]+) \[(.*?)\] "(.*?)" ([^\s]+) ([^\s]+)', line).groups()
            if tup[5] != "200":
                continue
            # (ip, userid)
            row = [None if x == '-' else x for x in [tup[3], tup[0], tup[2], tup[4], tup[6]]]
            row[3] = row[3].split()[1]
            #print(row)
            database_cur.execute("INSERT INTO logs VALUES (%s, %s, %s, %s, %s)", row)
    
    database_cur.close()
    database_conn.commit()

    

In [ ]:
# read in files
cur.execute("DROP TABLE IF EXISTS logs")

for i in range(len(log_files)):
    print(f"converting log {i} to sql")
    apache_log_to_sql(f'log{i}', conn, overwrite=False)

In [ ]:
def create_locations(ip_to_id: str, id_to_name: str, sql_conn):
    
    cur = sql_conn.cursor()
    cur.execute("DROP TABLE IF EXISTS locations")
    cur.execute('''CREATE TABLE locations
               (network inet PRIMARY KEY, country text)''')
    
    cur.close()
    sql_conn.commit()
    
    # change to db connection
    engine = create_engine('postgresql://zack@localhost:5432/payne_logs')

    ip_to_id = pd.read_csv(ip_to_id)
    ip_to_id.fillna(0, inplace=True)
    ip_to_id.geoname_id = ip_to_id.geoname_id.astype(int)
    id_to_name = pd.read_csv(id_to_name)


    cidr_to_name = pd.merge(ip_to_id, id_to_name, on="geoname_id")
    cidr_to_name['country'] = cidr_to_name['country_name']
    cidr_to_name = cidr_to_name[['network', 'country']]

    cidr_to_name.to_sql('locations', engine, if_exists='append', index=False)
    
    cur = sql_conn.cursor()
    #cur.execute('alter table locations add EXCLUDE USING gist (network inet_ops WITH &&)')
    cur.close()
    sql_conn.commit()


In [ ]:
create_locations('ip_to_id.csv', 'id_to_name.csv', conn)

In [ ]:
# test locations
engine = create_engine('postgresql://zack@localhost:5432/payne_logs')
locs = pd.read_sql("select (host(network)::inet - '0.0.0.0'::inet) as start, (host(broadcast(network))::inet - '0.0.0.0'::inet) as end, country from locations", engine)
locs

In [ ]:
def users_to_sql(users_file: str, conn):

    cur = conn.cursor()

    cur.execute("DROP TABLE IF EXISTS userid_to_id")
    cur.execute('''CREATE TABLE IF NOT EXISTS userid_to_id
               (userid text PRIMARY KEY, email text, id text, first text, last text)''')
    
    special_users = [
        "eognrt",
        "seavision",
        "embassy",
"oceana",
"student",
"gfw",
"irma",
"cmatthieu",
"oceanmind",
"ausdod",
"smarthub",
"fimo",
"khang",
"adi_susanto",
"bfar",
"phlcg",
"phlncwc",
"ana.sequeria",
"dws",
"m.meekan",
"carlos.duarte",
"ana.sequeira",
"imarpe"
    ]
    
    for user in special_users:
        row = (user, None, user, user, user)
        cur.execute("INSERT INTO userid_to_id VALUES (%s, %s, %s, %s, %s)", row)

    with open(users_file) as f:
        users = json.load(f)
        for user in users:
            #print(user)
            row = (user.get('id'), user.get('email'), user.get('id'), user.get('firstName'), user.get('lastName'))
            row_alt = (user.get('email'), user.get('email'), user.get('id'), user.get('firstName'), user.get('lastName'))
            cur.execute("INSERT INTO userid_to_id VALUES (%s, %s, %s, %s, %s)", row)
            if row_alt[0] is not None:
                cur.execute("INSERT INTO userid_to_id VALUES (%s, %s, %s, %s, %s)", row_alt)
            
    cur.close()

    conn.commit()

In [ ]:
users_to_sql("user_list.json", conn)

In [ ]:
def org_to_sql(ip_to_org, sql_conn):
    cur = sql_conn.cursor()
    cur.execute("DROP TABLE IF EXISTS isps")
    cur.execute('''CREATE TABLE isps
               (network inet PRIMARY KEY, isp text)''')
    
    cur.close()
    sql_conn.commit()
    
    engine = create_engine('postgresql://zack@localhost:5432/payne_logs')
    
    ip_to_org = pd.read_csv(ip_to_org)

    ip_to_org = ip_to_org[['network', 'isp']]

    ip_to_org.to_sql('isps', engine, if_exists='append', index=False)
    
    cur = sql_conn.cursor()
    # this takes a long time, uncomment if needed or run manually
    #cur.execute('alter table orgs add EXCLUDE USING gist (network inet_ops WITH &&)')
    cur.close()
    sql_conn.commit()
    

In [ ]:
org_to_sql("ip_to_org.csv", conn)

In [ ]:
cur.close()

conn.close()